### Dogs Image Research Inference
Esse notebook foi feito para testar os modelos de Dog breed comparando a acuracia de cada um sob o mesmo dataset, servindo assim para comparar o resultado diante de diferentes arquiteturas porem do mesmo input size. Ao final do notebook o melhor modelo baseado na acuracia do dataset de teste eh convertido para CPU e o formato ONNX para facilitar a inferencia pelo OpenCV.

by: Crystal Silva Campos <https://github.com/campos537>

## Step 1 
Importa as bibliotecas necessarias

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
# Plot library
import matplotlib.pyplot as plt
# Metric generation library based on confusion matrix
from pycm import *
# Fast array processing library
import numpy as np
# Deep Learning framework
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import cv2
import time

## Step 2
Aqui se escolhe o tamanho do batch de teste e o input size em que os modelos vao ser testados

In [2]:
models_path = "models/"
test_dir = "test_cleaned"


input_size = (224,224)
batch_size = 256
device = torch.device("cuda")

## Step 3
Aqui eh onde se carrega o dataset de teste e que eh feito o pre-processamento nas imagens

In [3]:
def load_test_dataset(input_size, test_dir, batch_size):
    test_transforms = transforms.Compose([transforms.Resize(input_size),
                                           transforms.ToTensor()
                                          ])
    test_data = datasets.ImageFolder(test_dir, transform = test_transforms)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle = True)
    return test_loader

## Step 4
Nesse metodo o modelo eh testado e se compara com o ground-truth esperado assim medindo a acuracia e retornando a mesma juntamente com outras informacoes que vao ser passadas ao PyCM

In [4]:
def test_network(loader, device_test, model):
    test_loss = 0
    accuracy = 0
    with torch.no_grad():
        count = 0
        actual_pred = []
        ground_truth = []
        for inputs, labels in loader:
            inputs, labels = inputs.to(device_test), labels.to(device_test)
            logps = model.forward(inputs)
            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim = 1)
            equals = top_class == labels.view(*top_class.shape)
            actual_pred.extend(top_class.squeeze().tolist())
            ground_truth.extend(labels.view(*top_class.squeeze().shape).tolist())
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            count += 1
    return (accuracy/len(loader)), actual_pred, ground_truth

## Step 5
Nesse passo cada modelo eh testado, as pastas devem estar divididas da seguinte forma:

models

    resnet50-dogbreed
    
        epoch1.pth
        
        epoch2.pth
        
        ...
    resnet34-dogbreed
    
        epoch1.pth
        
        epoch2.pth
        
        ....

In [ ]:
test_loader = load_test_dataset(input_size, test_dir, batch_size)

cm = None
best_accuracy_path = ["-",-999.0]
best_model = None

for model_type_path in os.listdir(models_path):
    
    models_list = os.listdir(models_path + '/' + model_type_path)
    models_list.sort()
    for model_path in models_list:
        full_model_path = models_path + '/' + model_type_path + '/' + model_path
        model_loaded = torch.load(full_model_path)
        model_loaded.eval()
        model_loaded.to(device)
        print("TESTING --> ", model_path)
        accuracy, actual_pred, ground_truth = test_network(test_loader,device,model_loaded)
        print(" ACCURACY --> ", accuracy , "%")
        if accuracy > best_accuracy_path[1]:
            best_accuracy_path[0] = model_path
            best_accuracy_path[1] = accuracy
            cm = ConfusionMatrix(actual_vector=ground_truth, predict_vector=actual_pred)
            best_model = model_loaded
    print("BEST MODEL INFO --> Type ", model_type_path, " Model ", best_accuracy_path[0], " Accuracy ", round(best_accuracy_path[1]*100,3), "%ACC\n\n")

TESTING -->  epoch_0_inputsize_224_224_batchsize_256_name_dog_breed_resnet50_.pt


/media/disk1/Repositorios/UnicoID/dogs-project/dogs-env/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


 ACCURACY -->  0.12197791039943695 %
TESTING -->  epoch_10_inputsize_224_224_batchsize_256_name_dog_breed_resnet50_.pt
 ACCURACY -->  0.881565979548863 %
TESTING -->  epoch_11_inputsize_224_224_batchsize_256_name_dog_breed_resnet50_.pt
 ACCURACY -->  0.9217228123119899 %
TESTING -->  epoch_12_inputsize_224_224_batchsize_256_name_dog_breed_resnet50_.pt
 ACCURACY -->  0.880428169454847 %
TESTING -->  epoch_13_inputsize_224_224_batchsize_256_name_dog_breed_resnet50_.pt
 ACCURACY -->  0.8919657383646283 %
TESTING -->  epoch_14_inputsize_224_224_batchsize_256_name_dog_breed_resnet50_.pt
 ACCURACY -->  0.8938065256391253 %
TESTING -->  epoch_15_inputsize_224_224_batchsize_256_name_dog_breed_resnet50_.pt
 ACCURACY -->  0.9081415278570992 %
TESTING -->  epoch_16_inputsize_224_224_batchsize_256_name_dog_breed_resnet50_.pt
 ACCURACY -->  0.9150263837405613 %
TESTING -->  epoch_17_inputsize_224_224_batchsize_256_name_dog_breed_resnet50_.pt
 ACCURACY -->  0.902017627443586 %
TESTING -->  epoch_18_

## Step 6
Apos testar o modelo utilizando o PyCM a matrix de confusao com os resultados serao gerados e um reporte do melhor modelo sera criado. Tendo em vista a grande quantidade de classes isso dificulta a visualizacao de cada metrica individual.

In [ ]:
print(cm.save_html(model_type_path))
cm.overall_stat

## Step 7
Converte o modelo para onnx de modo a facilitar a inferencia com o OpenCV DNN e tambem a otimizacao para o Openvino caso seja feita

In [ ]:
best_model.fc.add_module("prob_out", nn.Softmax())
best_model.eval()
best_model.to("cpu")
output_onnx_name = best_accuracy_path[0][:-3] + ".onnx"
dummy_input = torch.randn(1, 3, input_size[0], input_size[1], device='cpu')
torch.onnx.export(best_model, dummy_input, output_onnx_name , verbose=True)

## Step 8
Checa se a conversao para modelo ONNX ocorreu corretamente

In [ ]:
import onnx
onnx_model = onnx.load(output_onnx_name)
onnx.checker.check_model(onnx_model)
print(onnx.helper.printable_graph(onnx_model.graph))

## Step 9
Testa uma imagem de um shitzu(shih) com o modelo convertido

In [ ]:
labels = test_loader.dataset.classes
def process_output(out):
        classId = out[0].argsort()[-5:][::-1]
        confidence = [out[0][id_]*100 for id_ in classId]
        return classId, confidence
    
img = cv2.imread("chewie1.png")
model = cv2.dnn.readNetFromONNX(output_onnx_name)
blob =  cv2.dnn.blobFromImage(img, 1/255, (224,224), (), True, False)
model.setInput(blob)
out = model.forward()
ids, proba = process_output(out)

count = 0
for breed in ids:
    print(labels[breed]," ", proba[count], "%")
    count +=1

plt.imshow(img)
plt.title('Shitzu')
plt.show()